In [80]:
# Import Library
import pandas as pd
import re


In [127]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait

from selenium.common.exceptions import TimeoutException



In [151]:
df=pd.DataFrame(columns=['Unnamed: 0_level_0_Player', 'Unnamed: 1_level_0_Game',
       'Unnamed: 2_level_0_Pts*', 'Passing_Att', 'Passing_Cmp', 'Passing_Yds',
       'Passing_TD', 'Passing_Int', 'Passing_2Pt', 'Rushing_Att',
       'Rushing_Yds', 'Rushing_TD', 'Rushing_2Pt', 'Receiving_Rec',
       'Receiving_Yds', 'Receiving_TD', 'Receiving_2Pt', 'Fumbles_FL',
       'Fumbles_TD'])
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

for i in range(2010, 2022):
    for j in range(1, 18):

        season = i
        week = j
        
        # Scrape web for stats
        url = f"https://www.footballdb.com/fantasy-football/index.html?pos=QB&yr={season}&wk={week}&key=48ca46aa7d721af4d58dccc0c249a1c4"

        PATH = "/Applications/chromedriver"
        driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
        wait = WebDriverWait(driver, 20)
        driver.get(url)

        wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='leftcol']/div[3]/table")))
        driver.execute_script("window.stop();")

        result = pd.read_html(driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table").get_attribute("outerHTML"))
        regex_result = driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table/tbody").get_attribute("outerHTML")

        # find player's team based on which team is bolded
        regex_teams = re.findall("<b>(.*?)</b>", regex_result)

        # flatten multiindex
        result = result[0]
        result.columns = result.columns.get_level_values(0) + '_' +  result.columns.get_level_values(1)

        # Set values for new columns
        result['team'] = regex_teams
        result['season'] = season
        result['week'] = week

        # concatenate to master df
        df = pd.concat([df, result], axis=0, ignore_index=True)

df

,Unnamed: 0_level_0_Player,Unnamed: 1_level_0_Game,Unnamed: 2_level_0_Pts*,Passing_Att,Passing_Cmp,Passing_Yds,Passing_TD,Passing_Int,Passing_2Pt,Rushing_Att,...,Rushing_2Pt,Receiving_Rec,Receiving_Yds,Receiving_TD,Receiving_2Pt,Fumbles_FL,Fumbles_TD,team,season,week
0,Peyton ManningP. Manning,IND@HOU,35.0,57,40,433,3,0,0,0,...,0,0,0,0,0,0,0,IND,2010.0,1.0
1,Tom BradyT. Brady,CIN@NE,28.0,35,25,258,3,0,0,0,...,0,0,0,0,0,0,0,NE,2010.0,1.0
2,Jay CutlerJ. Cutler,DET@CHI,25.0,35,23,372,2,1,0,5,...,0,0,0,0,0,1,0,CHI,2010.0,1.0
3,David GarrardD. Garrard,DEN@JAX,25.0,21,16,170,3,0,0,7,...,0,0,0,0,0,0,0,JAX,2010.0,1.0
4,Carson PalmerC. Palmer,CIN@NE,23.0,50,34,345,2,1,0,4,...,0,0,0,0,0,0,0,CIN,2010.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7213,Jordan LoveJ. Love,MIN@GB,0.0,4,2,19,0,0,0,2,...,0,0,0,0,0,0,0,GB,2021.0,17.0
7214,Kellen MondK. Mond,MIN@GB,0.0,3,2,5,0,0,0,0,...,0,0,0,0,0,0,0,MIN,2021.0,17.0
7215,Cam NewtonC. Newton,CAR@NO,0.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,CAR,2021.0,17.0
7216,Brett RypienB. Rypien,DEN@LAC,0.0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,DEN,2021.0,17.0


In [152]:
df.to_csv('data_raw/weekly_qb.csv')

In [154]:
df=pd.DataFrame(columns=['Unnamed: 0_level_0_Player', 'Unnamed: 1_level_0_Game',
       'Unnamed: 2_level_0_Pts*', 'Passing_Att', 'Passing_Cmp', 'Passing_Yds',
       'Passing_TD', 'Passing_Int', 'Passing_2Pt', 'Rushing_Att',
       'Rushing_Yds', 'Rushing_TD', 'Rushing_2Pt', 'Receiving_Rec',
       'Receiving_Yds', 'Receiving_TD', 'Receiving_2Pt', 'Fumbles_FL',
       'Fumbles_TD'])
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

for i in range(2010, 2022):
    for j in range(1, 18):

        season = i
        week = j
        
        # Scrape web for stats
        url = f"https://www.footballdb.com/fantasy-football/index.html?pos=RB&yr={season}&wk={week}&key=48ca46aa7d721af4d58dccc0c249a1c4"

        PATH = "/Applications/chromedriver"
        driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
        wait = WebDriverWait(driver, 20)
        driver.get(url)

        wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='leftcol']/div[3]/table")))
        driver.execute_script("window.stop();")

        result = pd.read_html(driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table").get_attribute("outerHTML"))
        regex_result = driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table/tbody").get_attribute("outerHTML")

        # find player's team based on which team is bolded
        regex_teams = re.findall("<b>(.*?)</b>", regex_result)

        # flatten multiindex
        result = result[0]
        result.columns = result.columns.get_level_values(0) + '_' +  result.columns.get_level_values(1)

        # Set values for new columns
        result['team'] = regex_teams
        result['season'] = season
        result['week'] = week

        # concatenate to master df
        df = pd.concat([df, result], axis=0, ignore_index=True)

df

,Unnamed: 0_level_0_Player,Unnamed: 1_level_0_Game,Unnamed: 2_level_0_Pts*,Passing_Att,Passing_Cmp,Passing_Yds,Passing_TD,Passing_Int,Passing_2Pt,Rushing_Att,...,Rushing_2Pt,Receiving_Rec,Receiving_Yds,Receiving_TD,Receiving_2Pt,Fumbles_FL,Fumbles_TD,team,season,week
0,Arian FosterA. Foster,IND@HOU,41.0,0,0,0,0,0,0,33,...,0,1,7,0,0,0,0,HOU,2010.0,1.0
1,Matt ForteM. Forte,DET@CHI,31.0,0,0,0,0,0,0,17,...,0,7,151,2,0,1,0,CHI,2010.0,1.0
2,Chris JohnsonC. Johnson,OAK@TEN,26.0,0,0,0,0,0,0,27,...,0,4,8,0,0,0,0,TEN,2010.0,1.0
3,Darren McFaddenD. McFadden,OAK@TEN,20.0,0,0,0,0,0,0,18,...,0,6,55,1,0,0,0,OAK,2010.0,1.0
4,Rashard MendenhallR. Mendenhall,ATL@PIT,19.0,0,0,0,0,0,0,22,...,0,2,15,0,0,0,0,PIT,2010.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17241,Adam PrenticeA. Prentice,CAR@NO,0.0,0,0,0,0,0,0,1,...,0,1,4,0,0,0,0,NO,2021.0,17.0
17242,Craig ReynoldsC. Reynolds,DET@SEA,0.0,0,0,0,0,0,0,4,...,0,1,9,0,0,0,0,DET,2021.0,17.0
17243,Benjamin SnellB. Snell,CLE@PIT,0.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,PIT,2021.0,17.0
17244,Derek WattD. Watt,CLE@PIT,0.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,PIT,2021.0,17.0


In [155]:
df.to_csv('data_raw/weekly_rb.csv')

In [157]:
df=pd.DataFrame(columns=['Unnamed: 0_level_0_Player', 'Unnamed: 1_level_0_Game',
       'Unnamed: 2_level_0_Pts*', 'Passing_Att', 'Passing_Cmp', 'Passing_Yds',
       'Passing_TD', 'Passing_Int', 'Passing_2Pt', 'Rushing_Att',
       'Rushing_Yds', 'Rushing_TD', 'Rushing_2Pt', 'Receiving_Rec',
       'Receiving_Yds', 'Receiving_TD', 'Receiving_2Pt', 'Fumbles_FL',
       'Fumbles_TD'])
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

for i in range(2010, 2022):
    for j in range(1, 18):

        season = i
        week = j
        
        # Scrape web for stats
        url = f"https://www.footballdb.com/fantasy-football/index.html?pos=WR&yr={season}&wk={week}&key=48ca46aa7d721af4d58dccc0c249a1c4"

        PATH = "/Applications/chromedriver"
        driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
        wait = WebDriverWait(driver, 20)
        driver.get(url)

        wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='leftcol']/div[3]/table")))
        driver.execute_script("window.stop();")

        result = pd.read_html(driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table").get_attribute("outerHTML"))
        regex_result = driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table/tbody").get_attribute("outerHTML")

        # find player's team based on which team is bolded
        regex_teams = re.findall("<b>(.*?)</b>", regex_result)

        # flatten multiindex
        result = result[0]
        result.columns = result.columns.get_level_values(0) + '_' +  result.columns.get_level_values(1)

        # Set values for new columns
        result['team'] = regex_teams
        result['season'] = season
        result['week'] = week

        # concatenate to master df
        df = pd.concat([df, result], axis=0, ignore_index=True)

df

,Unnamed: 0_level_0_Player,Unnamed: 1_level_0_Game,Unnamed: 2_level_0_Pts*,Passing_Att,Passing_Cmp,Passing_Yds,Passing_TD,Passing_Int,Passing_2Pt,Rushing_Att,...,Rushing_2Pt,Receiving_Rec,Receiving_Yds,Receiving_TD,Receiving_2Pt,Fumbles_FL,Fumbles_TD,team,season,week
0,Hakeem NicksH. Nicks,CAR@NYG,25.0,0,0,0,0,0,0,0,...,0,4,75,3,0,0,0,NYG,2010.0,1.0
1,Austin CollieA. Collie,IND@HOU,21.0,0,0,0,0,0,0,0,...,0,11,163,1,0,1,0,IND,2010.0,1.0
2,Chad JohnsonC. Johnson,CIN@NE,21.0,0,0,0,0,0,0,0,...,0,12,159,1,0,0,0,CIN,2010.0,1.0
3,Miles AustinM. Austin,DAL@WAS,20.0,0,0,0,0,0,0,0,...,0,10,146,1,0,0,0,DAL,2010.0,1.0
4,Wes WelkerW. Welker,CIN@NE,18.0,0,0,0,0,0,0,0,...,0,8,64,2,0,0,0,NE,2010.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21742,Jalen ReagorJ. Reagor,PHI@WAS,0.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,PHI,2021.0,17.0
21743,Marquez Valdes-ScantlingM. Valdes-Scantling,MIN@GB,0.0,0,0,0,0,0,0,0,...,0,1,3,0,0,0,0,GB,2021.0,17.0
21744,Dede WestbrookD. Westbrook,MIN@GB,0.0,0,0,0,0,0,0,0,...,0,1,4,0,0,0,0,MIN,2021.0,17.0
21745,Olamide ZaccheausO. Zaccheaus,ATL@BUF,0.0,0,0,0,0,0,0,0,...,0,2,8,0,0,0,0,ATL,2021.0,17.0


In [158]:
df.to_csv('data_raw/weekly_wr.csv')

In [149]:
df=pd.DataFrame(columns=['Unnamed: 0_level_0_Player', 'Unnamed: 1_level_0_Game',
       'Unnamed: 2_level_0_Pts*', 'Passing_Att', 'Passing_Cmp', 'Passing_Yds',
       'Passing_TD', 'Passing_Int', 'Passing_2Pt', 'Rushing_Att',
       'Rushing_Yds', 'Rushing_TD', 'Rushing_2Pt', 'Receiving_Rec',
       'Receiving_Yds', 'Receiving_TD', 'Receiving_2Pt', 'Fumbles_FL',
       'Fumbles_TD'])
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

for i in range(2010, 2022):
    for j in range(1, 18):

        season = i
        week = j
        
        # Scrape web for stats
        url = f"https://www.footballdb.com/fantasy-football/index.html?pos=TE&yr={season}&wk={week}&key=48ca46aa7d721af4d58dccc0c249a1c4"

        PATH = "/Applications/chromedriver"
        driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
        wait = WebDriverWait(driver, 20)
        driver.get(url)

        wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='leftcol']/div[3]/table")))
        driver.execute_script("window.stop();")

        result = pd.read_html(driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table").get_attribute("outerHTML"))
        regex_result = driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table/tbody").get_attribute("outerHTML")

        # find player's team based on which team is bolded
        regex_teams = re.findall("<b>(.*?)</b>", regex_result)

        # flatten multiindex
        result = result[0]
        result.columns = result.columns.get_level_values(0) + '_' +  result.columns.get_level_values(1)

        # Set values for new columns
        result['team'] = regex_teams
        result['season'] = season
        result['week'] = week

        # concatenate to master df
        df = pd.concat([df, result], axis=0, ignore_index=True)

df

,Unnamed: 0_level_0_Player,Unnamed: 1_level_0_Game,Unnamed: 2_level_0_Pts*,Passing_Att,Passing_Cmp,Passing_Yds,Passing_TD,Passing_Int,Passing_2Pt,Rushing_Att,...,Rushing_2Pt,Receiving_Rec,Receiving_Yds,Receiving_TD,Receiving_2Pt,Fumbles_FL,Fumbles_TD,team,season,week
0,Marcedes LewisM. Lewis,DEN@JAX,15.0,0,0,0,0,0,0,0,...,0,2,31,2,0,0,0,JAX,2010.0,1.0
1,Dallas ClarkD. Clark,IND@HOU,14.0,0,0,0,0,0,0,0,...,0,11,80,1,0,0,0,IND,2010.0,1.0
2,Antonio GatesA. Gates,SD@KC,13.0,0,0,0,0,0,0,0,...,0,5,76,1,0,0,0,SD,2010.0,1.0
3,Visanthe ShiancoeV. Shiancoe,MIN@NO,13.0,0,0,0,0,0,0,0,...,0,4,76,1,0,0,0,MIN,2010.0,1.0
4,Bo ScaifeB. Scaife,OAK@TEN,9.0,0,0,0,0,0,0,0,...,0,3,37,1,0,0,0,TEN,2010.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10316,Juwan JohnsonJ. Johnson,CAR@NO,0.0,0,0,0,0,0,0,0,...,0,1,4,0,0,0,0,NO,2021.0,17.0
10317,Brevin JordanB. Jordan,HOU@SF,0.0,0,0,0,0,0,0,0,...,0,1,9,0,0,0,0,HOU,2021.0,17.0
10318,Foster MoreauF. Moreau,LV@IND,0.0,0,0,0,0,0,0,0,...,0,1,9,0,0,0,0,LV,2021.0,17.0
10319,Albert OkwuegbunamA. Okwuegbunam,DEN@LAC,0.0,0,0,0,0,0,0,0,...,0,2,5,0,0,0,0,DEN,2021.0,17.0


In [150]:
df.to_csv('data_raw/weekly_te.csv')

In [139]:
df=pd.DataFrame(columns=['Player', 'Game', 'Pts*', 'XPA', 'XPM', 'FGA', 'FGM', '50+'])
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

for i in range(2010, 2022):
    for j in range(1, 18):

        season = i
        week = j
        
        # Scrape web for stats
        url = f"https://www.footballdb.com/fantasy-football/index.html?pos=K&yr={season}&wk={week}&key=48ca46aa7d721af4d58dccc0c249a1c4"

        PATH = "/Applications/chromedriver"
        driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
        wait = WebDriverWait(driver, 20)
        driver.get(url)

        wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='leftcol']/div[3]/table")))
        driver.execute_script("window.stop();")

        result = pd.read_html(driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table").get_attribute("outerHTML"))
        regex_result = driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table/tbody").get_attribute("outerHTML")

        # find player's team based on which team is bolded
        regex_teams = re.findall("<b>(.*?)</b>", regex_result)

        # flatten multiindex
        result = result[0]
        #result.columns = result.columns.get_level_values(0) + '_' +  result.columns.get_level_values(1)

        # Set values for new columns
        result['team'] = regex_teams
        result['season'] = season
        result['week'] = week

        # concatenate to master df
        df = pd.concat([df, result], axis=0, ignore_index=True)

df

,Player,Game,Pts*,XPA,XPM,FGA,FGM,50+,team,season,week
0,John KasayJ. Kasay,CAR@NYG,12.0,1,1,3,3,1,CAR,2010.0,1.0
1,Mason CrosbyM. Crosby,GB@PHI,11.0,3,3,2,2,1,GB,2010.0,1.0
2,Jeff ReedJ. Reed,ATL@PIT,11.0,0,0,5,3,1,ATL,2010.0,1.0
3,Neil RackersN. Rackers,IND@HOU,10.0,4,4,2,2,0,HOU,2010.0,1.0
4,Matt BryantM. Bryant,ATL@PIT,9.0,0,0,4,3,0,ATL,2010.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
6068,Jason SandersJ. Sanders,MIA@TEN,3.0,0,0,2,1,0,MIA,2021.0,17.0
6069,Matt GayM. Gay,LA@BAL,2.0,2,2,1,0,0,LA,2021.0,17.0
6070,Chase McLaughlinC. McLaughlin,CLE@PIT,2.0,2,2,0,0,0,CLE,2021.0,17.0
6071,Greg ZuerleinG. Zuerlein,ARI@DAL,2.0,2,2,1,0,0,DAL,2021.0,17.0


In [140]:
df.to_csv('data_raw/weekly_kicker.csv')

In [144]:
df=pd.DataFrame(columns=['Team', 'Game', 'Pts*', 'Sack', 'Int', 'Saf', 'FR', 'Blk', 'TD', 'PA',
       'PassYds', 'RushYds', 'TotYds'])
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

for i in range(2010, 2022):
    for j in range(1, 18):

        season = i
        week = j
        
        # Scrape web for stats
        url = f"https://www.footballdb.com/fantasy-football/index.html?pos=DST&yr={season}&wk={week}&key=48ca46aa7d721af4d58dccc0c249a1c4"

        PATH = "/Applications/chromedriver"
        driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
        wait = WebDriverWait(driver, 20)
        driver.get(url)

        wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='leftcol']/div[3]/table")))
        driver.execute_script("window.stop();")

        result = pd.read_html(driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table").get_attribute("outerHTML"))
        regex_result = driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table/tbody").get_attribute("outerHTML")

        # find player's team based on which team is bolded
        regex_teams = re.findall("<b>(.*?)</b>", regex_result)

        # flatten multiindex
        result = result[0]
        #result.columns = result.columns.get_level_values(0) + '_' +  result.columns.get_level_values(1)

        # Set values for new columns
        result['team'] = regex_teams
        result['season'] = season
        result['week'] = week

        # concatenate to master df
        df = pd.concat([df, result], axis=0, ignore_index=True)

df



,Team,Game,Pts*,Sack,Int,Saf,FR,Blk,TD,PA,PassYds,RushYds,TotYds,team,season,week
0,Seattle SeahawksSEA,SF@SEA,22.0,2.0,2,0,0,0,1,6,214,49,263,SEA,2010.0,1.0
1,Arizona CardinalsARI,ARI@STL,19.0,2.0,3,0,1,1,0,13,240,85,325,ARI,2010.0,1.0
2,New York GiantsNYG,CAR@NYG,18.0,4.0,3,0,2,0,0,18,148,89,237,NYG,2010.0,1.0
3,New England PatriotsNE,CIN@NE,18.0,1.0,1,0,1,0,2,24,341,87,428,NE,2010.0,1.0
4,Detroit LionsDET,DET@CHI,16.0,4.0,1,0,3,0,0,19,362,101,463,DET,2010.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6139,Miami DolphinsMIA,MIA@TEN,1.0,1.0,0,0,0,0,0,34,110,198,308,MIA,2021.0,17.0
6140,Detroit LionsDET,DET@SEA,1.0,1.0,0,0,0,0,0,51,232,265,497,DET,2021.0,17.0
6141,Denver BroncosDEN,DEN@LAC,0.0,0.0,0,0,0,0,0,34,237,104,341,DEN,2021.0,17.0
6142,Minnesota VikingsMIN,MIN@GB,0.0,0.0,0,0,0,0,0,37,307,174,481,MIN,2021.0,17.0


In [145]:
df.to_csv('data_raw/weekly_defense.csv')